In [10]:
import akshare as ak

# 获取期货数据
futures_zh_minute_sina_df = ak.futures_zh_minute_sina(symbol="FG2501", period="5")

# 选择需要的列，并重命名列标题为中文
futures_zh_minute_sina_df = futures_zh_minute_sina_df[['时间', 'open', 'high', 'low', '收盘价']]
futures_zh_minute_sina_df.columns = ['时间', '开盘价', '最高价', '最低价', '收盘价']

# 将数据保存到当前目录下的 CSV 文件
futures_zh_minute_sina_df.to_csv("玻璃期货五分钟行情数据.csv", index=False)

print("数据已保存到当前目录下的 五分钟行情数据.csv 文件中，标题为中文")


数据已保存到当前目录下的 五分钟行情数据.csv 文件中，标题为中文


In [26]:
import akshare as ak
import pandas as pd

# 获取期货数据
futures_zh_minute_sina_df = ak.futures_zh_minute_sina(symbol="FG2501", period="5")

# 选择需要的列
futures_zh_minute_sina_df = futures_zh_minute_sina_df[['datetime', 'close']]

# 转换 datetime 列为日期时间格式
futures_zh_minute_sina_df['datetime'] = pd.to_datetime(futures_zh_minute_sina_df['datetime'])

# 计算 MACD 参数 (12, 26, 9)
# 快速移动平均线（12 天 EMA）
futures_zh_minute_sina_df['EMA12'] = futures_zh_minute_sina_df['close'].ewm(span=12, adjust=False).mean()

# 慢速移动平均线（26 天 EMA）
futures_zh_minute_sina_df['EMA26'] = futures_zh_minute_sina_df['close'].ewm(span=26, adjust=False).mean()

# DIF = EMA12 - EMA26
futures_zh_minute_sina_df['DIF'] = futures_zh_minute_sina_df['EMA12'] - futures_zh_minute_sina_df['EMA26']

# DEA（即 Signal 线）= DIF 的 9 日 EMA
futures_zh_minute_sina_df['DEA'] = futures_zh_minute_sina_df['DIF'].ewm(span=9, adjust=False).mean()

# 保留 datetime, close, 和 DEA 列，并将 DEA 四舍五入为整数
futures_zh_minute_sina_df = futures_zh_minute_sina_df[['datetime', 'close', 'DEA']]
futures_zh_minute_sina_df['DEA'] = futures_zh_minute_sina_df['DEA'].round(0).astype(int)

# 重命名列标题为中文
futures_zh_minute_sina_df.columns = ['时间', '收盘价', 'DEA']

# 保存到当前目录
futures_zh_minute_sina_df.to_csv("玻璃期货五分钟行情数据DEA.csv", index=False)

print("数据已保存到当前目录下的 玻璃期货五分钟行情数据DEA.csv 文件中，仅包含时间、收盘价和四舍五入为整数的 DEA")


数据已保存到当前目录下的 玻璃期货五分钟行情数据DEA.csv 文件中，仅包含时间、收盘价和四舍五入为整数的 DEA


In [34]:
import pandas as pd

# 读取保存的 CSV 文件
futures_zh_minute_sina_df = pd.read_csv("玻璃期货五分钟行情数据DEA.csv")

# 确保数据中的时间列是 datetime 格式
futures_zh_minute_sina_df['时间'] = pd.to_datetime(futures_zh_minute_sina_df['时间'])

# 过滤掉10月份的数据
futures_zh_minute_sina_df = futures_zh_minute_sina_df[futures_zh_minute_sina_df['时间'].dt.month != 10]

# 创建一个空的 DataFrame 来保存符合条件的数据
result_df = pd.DataFrame(columns=futures_zh_minute_sina_df.columns)

# 计算 DEA 是否由负转正，并输出相应的数据
for i in range(1, len(futures_zh_minute_sina_df)):
    # 检查当前 DEA 和前一条 DEA 是否发生了由负到正的变化
    if futures_zh_minute_sina_df['DEA'].iloc[i-1] < 0 and futures_zh_minute_sina_df['DEA'].iloc[i] == 0:
        # 将符合条件的数据行添加到 result_df
        result_df = pd.concat([result_df, futures_zh_minute_sina_df.iloc[[i]]], ignore_index=True)

# 将结果保存到 CSV 文件
result_df.to_csv("DEA由负转正的数据.csv", index=False)

print("符合条件的数据已保存到 'DEA由负转正的数据.csv' 文件中。")


符合条件的数据已保存到 'DEA由负转正的数据.csv' 文件中。


C:\Users\WangRuihua\AppData\Local\Temp\ipykernel_21620\3351564772.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, futures_zh_minute_sina_df.iloc[[i]]], ignore_index=True)


In [1]:
import pandas as pd

# 读取保存的 CSV 文件
futures_zh_minute_sina_df = pd.read_csv("玻璃期货五分钟行情数据DEA.csv")

# 确保数据中的时间列是 datetime 格式
futures_zh_minute_sina_df['时间'] = pd.to_datetime(futures_zh_minute_sina_df['时间'])

# 过滤掉10月份的数据
futures_zh_minute_sina_df = futures_zh_minute_sina_df[futures_zh_minute_sina_df['时间'].dt.month != 10]

# 创建一个空的 DataFrame 来保存符合条件的数据
result_df_negative_to_positive = pd.DataFrame(columns=futures_zh_minute_sina_df.columns)

# 计算 DEA 是否由正转负，并输出相应的数据
for i in range(1, len(futures_zh_minute_sina_df)):
    # 检查当前 DEA 和前一条 DEA 是否发生了由正到负的变化
    if futures_zh_minute_sina_df['DEA'].iloc[i-1] == 0 and futures_zh_minute_sina_df['DEA'].iloc[i] < 0:
        # 将符合条件的数据行添加到 result_df
        result_df_negative_to_positive = pd.concat([result_df_negative_to_positive, futures_zh_minute_sina_df.iloc[[i]]], ignore_index=True)

# 将结果保存到 CSV 文件
result_df_negative_to_positive.to_csv("DEA由正转负的数据.csv", index=False)

print("符合条件的数据已保存到 'DEA由正转负的数据.csv' 文件中。")


符合条件的数据已保存到 'DEA由正转负的数据.csv' 文件中。


C:\Users\WangRuihua\AppData\Local\Temp\ipykernel_30764\861205004.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df_negative_to_positive = pd.concat([result_df_negative_to_positive, futures_zh_minute_sina_df.iloc[[i]]], ignore_index=True)
